In [7]:
from collections import *

def train_char_lm(fname, order=4):
    f = open(fname,'r')
    data = f.read()
    lm = defaultdict(Counter)
#    pad = "~" * order
#    data = pad + data
    data = data.split('\n')
    for word in data:
        for i in range(len(word)-1):
            history, char = word[i:i+order], word[i+order]
            lm[history][char]+=1
    def normalize(counter):
        s = float(sum(counter.values()))
        return [(c,cnt/s) for c,cnt in counter.items()]
    outlm = {hist:normalize(chars) for hist, chars in lm.items()}
    return outlm

In [9]:
from random import random

def generate_letter(lm, history, order):
        history = history[-order:]
        dist = lm[history]
        x = random()
        for c,v in dist:
            x = x - v
            if x <= 0: return c

In [34]:
from __future__ import print_function 

def hangman(lm, secret_word, guesser, max_mistakes=11, verbose=True):
    secret_word = secret_word.lower()
    n = len(secret_word)
    mask = ['_'] * len(secret_word)
    guessed = set()
    if verbose:
        print("Starting hangman game. Target is", ' '.join(mask), 'length', len(secret_word))
    
    mistakes = 0
    while mistakes < max_mistakes:
        if verbose:
            print("You have", (max_mistakes-mistakes), "attempts remaining.")
        guess = guesser(mask, guessed, n, lm)

        if verbose:
            print('Guess is', guess)
        if guess in guessed:
            if verbose:
                print('Already guessed this before.')
            mistakes += 1
        else:
            try:
                guessed.add(guess)
            except:
                print(guessed,guess)
            if guess in secret_word:
                for i, c in enumerate(secret_word):
                    if c == guess:
                        mask[i] = c
                if verbose:
                    print('Good guess:', ' '.join(mask))
            else:
                if verbose:
                    print('Sorry, try again.')
                mistakes += 1
                
        if '_' not in mask:
            if verbose:
                print('Congratulations, you won.')
            return mistakes
        
    if verbose:
        print('Out of guesses. The word was', secret_word)    
    return mistakes


In [8]:
lm = train_char_lm("corncob_lowercase.txt",order=1)

In [28]:
def maxLi(mask,guessed,nword,lm):
    if mask.count('_') == nword:
        for c in ['e','i','a','o','u']:
            if c not in guessed:
                return c
    else:
        for i in range(nword):
            if mask[i] == '_':
                i = i+1
            else: 
                break
        hist = mask[i]
        c = generate_letter(lm,hist,len(hist))
        return c

In [36]:
hangman(lm,'abandon',maxLi)

Starting hangman game. Target is _ _ _ _ _ _ _ length 7
You have 11 attempts remaining.
Guess is e
Sorry, try again.
You have 10 attempts remaining.
Guess is i
Sorry, try again.
You have 9 attempts remaining.
Guess is a
Good guess: a _ a _ _ _ _
You have 9 attempts remaining.
Guess is n
Good guess: a _ a n _ _ n
You have 9 attempts remaining.
Guess is t
Sorry, try again.
You have 8 attempts remaining.
Guess is t
Already guessed this before.
You have 7 attempts remaining.
Guess is e
Already guessed this before.
You have 6 attempts remaining.
Guess is n
Already guessed this before.
You have 5 attempts remaining.
Guess is n
Already guessed this before.
You have 4 attempts remaining.
Guess is s
Sorry, try again.
You have 3 attempts remaining.
Guess is l
Sorry, try again.
You have 2 attempts remaining.
Guess is g
Sorry, try again.
You have 1 attempts remaining.
Guess is g
Already guessed this before.
Out of guesses. The word was abandon


11

In [ ]:
def generate_text(lm, order, nletters=1000):
    history = "~" * order
    out = []
    for i in range(nletters):
        c = generate_letter(lm, history, order)
        history = history[-order:] + c
        out.append(c)
    return "".join(out)

[('a', 0.0003893972686563013),
 ('r', 0.11412121380691459),
 ('b', 0.051150112647067),
 ('c', 0.06024532027925347),
 ('f', 0.009401162628987846),
 ('l', 0.12930770728451033),
 ('n', 0.12449586960754318),
 ('s', 0.061719467082023756),
 ('t', 0.18006842266292103),
 ('m', 0.03752120824409646),
 ('z', 0.004505882680165772),
 ('d', 0.04085889911829333),
 ('h', 0.002252941340082886),
 ('e', 0.004227741773982699),
 ('i', 0.03443384418546436),
 ('p', 0.03568547826328818),
 ('g', 0.03117959558312241),
 ('k', 0.012822295775039636),
 ('u', 0.018134787083136317),
 ('v', 0.016744082552220953),
 ('y', 0.014769282118321141),
 ('j', 0.0006397240842210664),
 ('x', 0.00392178677718132),
 ('o', 0.000723166356075988),
 ('q', 0.0006953522654576809),
 ('w', 0.009985258531972296)]